In [92]:
# import packages
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential

from hyperas import optim
from hyperas.distributions import choice, uniform

# Basics
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

# Graphs
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

# Filter warnings
import warnings
warnings.filterwarnings("ignore")

# Preprocessing; model selection and evaluation
from sklearn import pipeline, preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import classification_report

# text handling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.neural_network import MLPClassifier
import statsmodels.api as sm

# for custom countvectorizer with SpaCy lemmatization
import spacy
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, VectorizerMixin
from sklearn.base import TransformerMixin, BaseEstimator
from scipy.sparse import csr_matrix

# WordCloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Tensorflow/RNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

------
If we need to move virtual ENV to use Tensorflow we will need to install:

1. spacy

<code> conda install -c spacy spacy </code>

2. 'en_core_web_md'

<code> python -m spacy download en_core_web_md </code>

3. wordcloud

<code> conda install -c conda-forge wordcloud </code>

------

In [2]:
# import packages
data = pd.read_csv("saved_csv/df.csv")
data.drop(columns = "Unnamed: 0",inplace=True)

df = data.copy()

### Setting up data for analysis
------

In [3]:
# putting questions into categories

current_mh_coverage = ["Does your employer provide mental health benefits as part of healthcare coverage?",
               "Do you know the options for mental health care available under your employer-provided health coverage?",
               "Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",
               "Does your employer offer resources to learn more about mental health disorders and options for seeking help?",
               "Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?",
               "If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?"]

previous_mh_coverage = ["Have your previous employers provided mental health benefits?",
                        "Were you aware of the options for mental health care provided by your previous employers?",
                        "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?",
                        "Did your previous employers provide resources to learn more about mental health disorders and how to seek help?",
                        "Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?"]

mh_status = ["Do you currently have a mental health disorder?",
             "Have you ever been diagnosed with a mental health disorder?",'Anxiety Disorder', 'Mood Disorder', 
             'Psychotic Disorder','Eating Disorder', 'Neurodevelopmental Disorders','Personality Disorder', 
             'Obsessive-Compulsive Disorder','Post-Traumatic Stress Disorder', 'Dissociative Disorder',
             'Substance-Related and Addictive Disorders', 'Other','Adjustment disorder',
             "Have you had a mental health disorder in the past?",
             "Have you ever sought treatment for a mental health disorder from a mental health professional?",
             "Do you have a family history of mental illness?",
             "How willing would you be to share with friends and family that you have a mental illness?",
             "Would you be willing to bring up a physical health issue with a potential employer in an interview?"]

witnessed_exp = ["Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?",
                 "Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?",
                 "Have you observed or experienced supportive or well handled response to a mental health issue in your current or previous workplace?"]

ratings = df.columns[df.columns.str.contains("Overall")]

demographics = ["What is your age?","What is your gender?","What country do you live in?",
                "What US state or territory do you live in?","What is your race?"]

comfort_talking_current = ["Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?",
                           "Have you ever discussed your mental health with your employer?",
                           "Would you feel comfortable discussing a mental health issue with your coworkers?",
                           "Have you ever discussed your mental health with coworkers?",
                           "Have you ever had a coworker discuss their or another coworker's mental health with you?",
                           "Would you feel more comfortable talking to your coworkers about your physical health or your mental health?",
                           "Would you bring up your mental health with a potential employer in an interview?",
                           "Are you openly identified at work as a person with a mental health issue?"]

comfort_talking_previous = ["Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?",
                            "Would you have been willing to discuss your mental health with your direct supervisor(s)?",
                            "Did you ever discuss your mental health with your previous employer?",
                            "Would you have been willing to discuss your mental health with your coworkers at previous employers?",
                            "Did you ever discuss your mental health with a previous coworker(s)?",
                            "Did you ever have a previous coworker discuss their or another coworker's mental health with you?",
                            "Would you bring up your mental health with a potential employer in an interview?",
                            "Are you openly identified at work as a person with a mental health issue?"]

comfort_dependent_var = ["Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?",
                           "Have you ever discussed your mental health with your employer?",
                           "Would you feel comfortable discussing a mental health issue with your coworkers?",
                           "Have you ever discussed your mental health with coworkers?",
                           "Have you ever had a coworker discuss their or another coworker's mental health with you?",
                           "Would you feel more comfortable talking to your coworkers about your physical health or your mental health?",
                           "Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?",
                           "Would you have been willing to discuss your mental health with your direct supervisor(s)?",
                           "Did you ever discuss your mental health with your previous employer?",
                           "Would you have been willing to discuss your mental health with your coworkers at previous employers?",
                           "Did you ever discuss your mental health with a previous coworker(s)?",
                           "Did you ever have a previous coworker discuss their or another coworker's mental health with you?",
                           "Would you bring up your mental health with a potential employer in an interview?",
                           "Are you openly identified at work as a person with a mental health issue?"]

categories = [current_mh_coverage,previous_mh_coverage,mh_status,witnessed_exp,ratings,comfort_talking_current,comfort_talking_previous,comfort_dependent_var]

### Preparing the data
------

#### Independent Variables
------

In [4]:
def make_dummies(question,columns_to_keep = 1):
    
    dummies = pd.get_dummies(df_2.loc[:,question])
    for j in range(len(dummies.columns)):
        name = question + "__" + dummies.columns[j]
        dummies.rename(columns = {dummies.columns[j] : name},inplace=True)

    result = dummies.iloc[:,[columns_to_keep]]
    
    return result

In [5]:
# making a copy
df_2 = df.copy()

In [6]:
# preparing a table of independent variables for Q1
independent_var = [current_mh_coverage,previous_mh_coverage,witnessed_exp,mh_status,ratings]
dependent_var = [comfort_talking_current,comfort_talking_previous]

In [7]:
# set up 2 dataframes for concatenating data
omitted = pd.DataFrame(columns = ["Question","Answer"])
final = pd.DataFrame()

In [8]:
# current_mh_coverage
for i in [0,2,3]:
    result = make_dummies(current_mh_coverage[i],-1)
    final = pd.concat([final,result],axis = 1)

for i in [2,-2,-1]:
    result = make_dummies(current_mh_coverage[5],i)
    final = pd.concat([final,result],axis = 1)

In [9]:
# witnessed_exp
result = make_dummies(witnessed_exp[0],-1)
final = pd.concat([final,result],axis = 1)

In [10]:
# mh_status
old_answer_1 = "Possibly"
old_answer_2 = "-1"
answer = "Don't Know"

to_dummy = []
for i in [0,1,-5,-3,-1]:
    to_dummy.append(mh_status[i])
    
df_2.loc[:,to_dummy[0]][df_2.loc[:,to_dummy[0]]==old_answer_1]=answer

df_2.loc[:,to_dummy[2]][df_2.loc[:,to_dummy[2]]==old_answer_1]=answer
df_2.loc[:,to_dummy[2]][df_2.loc[:,to_dummy[2]]==old_answer_2]=answer

In [11]:
# mh_status
result = make_dummies(to_dummy[0],-1)
final = pd.concat([final,result],axis = 1)

for i in [2,3,4,5,6,7,8,9,10,11,12,13,15,17]:
    final = pd.concat([final,df_2.loc[:,mh_status[i]]],axis = 1)

In [12]:
# ratings
for i in [1,4]:
    final = pd.concat([final,df_2.loc[:,ratings[i]]],axis = 1)

In [13]:
independent_q1 = final

#### Dependent Variables
------

In [14]:
# grabbing the dependent variables
dependent_class = pd.read_csv("saved_csv/q1_dependent_alt.csv")
dependent_class.drop('Unnamed: 0',axis=1,inplace=True)

In [15]:
# simplifing dependent variables to 0/1
dependent_class_alt = dependent_class.copy()

dependent_class_alt[dependent_class_alt < 2] = 0
dependent_class_alt[dependent_class_alt >= 2] = 1

### Creating a model to predict comfort level using text responses
------

In [16]:
# Code source: https://github.com/mpavlovic/spacy-vectorizers
# create a custom countvectorizer with SpaCy lemmatization

class SpacyPipeInitializer(object):
    def __init__(self, nlp, join_str=" ", batch_size=10000, n_threads=2):
        self.nlp = nlp
        self.join_str = join_str
        self.batch_size = batch_size
        self.n_threads = n_threads
        
class SpacyPipeProcessor(SpacyPipeInitializer):
    def __init__(self, nlp, multi_iters=False, join_str=" ", batch_size=10000, n_threads=2):
        super(SpacyPipeProcessor, self).__init__(nlp, join_str, batch_size, n_threads)
        self.multi_iters = multi_iters
    
    def __call__(self, raw_documents):
        docs_generator = self.nlp.pipe(raw_documents, batch_size=self.batch_size, n_threads=self.n_threads)
        return docs_generator if self.multi_iters == False else list(docs_generator)
    
class SpacyLemmaCountVectorizer(CountVectorizer):
    
    def __init__(self, input='content', encoding='utf-8',
                 decode_error='strict', strip_accents=None,
                 lowercase=True, preprocessor=None, tokenizer=None,
                 stop_words=None, token_pattern=r"(?u)[^\r\n ]+",
                 ngram_range=(1, 1), analyzer='word',
                 max_df=1.0, min_df=1, max_features=None,
                 vocabulary=None, binary=False, dtype=np.int64, 
                 nlp=None, ignore_chars='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~', 
                 join_str=" ", use_pron=False):
        
        super().__init__(input, encoding, decode_error, strip_accents, 
                                                   lowercase, preprocessor, tokenizer,
                                                   stop_words, token_pattern, ngram_range, 
                                                   analyzer, max_df, min_df, max_features,
                                                   vocabulary, binary, dtype)
        self.ignore_chars = ignore_chars
        self.join_str = ' ' # lemmas have to be joined for splitting
        self.use_pron = use_pron
        self.translate_table = dict((ord(char), None) for char in self.ignore_chars)
        
    def lemmatize_from_docs(self, docs):
        for doc in docs:
            lemmas_gen = (token.lemma_.translate(self.translate_table) if self.use_pron or token.lemma_!='-PRON-' else token.lower_.translate(self.translate_table) for token in doc)  # generator expression
            yield self.join_str.join(lemmas_gen) if self.join_str is not None else [lemma for lemma in lemmas_gen]
    
    def build_tokenizer(self):
        return lambda doc: doc.split()
    
    def transform(self, spacy_docs):
        raw_documents = self.lemmatize_from_docs(spacy_docs)
        return super(SpacyLemmaCountVectorizer, self).transform(raw_documents)
    
    def fit_transform(self, spacy_docs, y=None):
        raw_documents = self.lemmatize_from_docs(spacy_docs)
        return super(SpacyLemmaCountVectorizer, self).fit_transform(raw_documents, y)

In [43]:
# Grabbing the responses as independent variables
corpus = df.iloc[:,-9]

# grabbing the dependent variables
dependent_class = pd.read_csv("saved_csv/q1_dependent_alt.csv")
dependent_class.drop('Unnamed: 0',axis=1,inplace=True)

In [44]:
# Creating a table with both independent and dependent variables
table = pd.concat([corpus,dependent_class_alt],axis=1)

# dropping columns that did not answer the question
index = table[table.iloc[:,0]=="Did not answer"].index

table.drop(index,axis=0,inplace=True)

In [19]:
# customization stopwords to filter out some words
stopwords = set(STOPWORDS)
stopwords.update(["mental","health","issue","work",
                  "take","hour","tech","industry","people","employee"])

# CountVectorizer with SpaCy Lemmatization
nlp = spacy.load('en_core_web_md')

spp = SpacyPipeProcessor(nlp, n_threads=1, multi_iters=True)
spacy_docs = spp(table.iloc[:,0]);

slcv = SpacyLemmaCountVectorizer(min_df=3,stop_words=stopwords, ngram_range=(1, 3), ignore_chars='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
slcv.fit(spacy_docs)
count_vectors = slcv.transform(spacy_docs); count_vectors

<789x755 sparse matrix of type '<class 'numpy.int64'>'
	with 7266 stored elements in Compressed Sparse Row format>

In [ ]:
# Baseline RNN model
# Code source for Early Stopping: 
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

X_rest, X_test, y_rest, y_test = train_test_split(count_vectors,table.iloc[:,1].values,test_size = 0.1)
X_train, X_validation, y_train, y_validation = train_test_split(X_rest,y_rest,test_size = 0.2)

X_train = X_train.toarray().reshape(568,755,1)
y_train = y_train.reshape(568,1)
X_validation = X_validation.toarray().reshape(142,755,1)
y_validation = y_validation.reshape(142,1)
X_test = X_test.toarray().reshape(79,755,1)
y_test = y_test.reshape(79,1)

model = Sequential()

model.add(LSTM(64,activation="relu", input_shape = (X_train.shape[1:]), return_sequences=True, dropout=0.2))
model.add(BatchNormalization())

model.add(LSTM(64, activation="relu"))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(2, activation="softmax"))

# setting up SGD (optimizer) hyperparameters
sgd = SGD(lr=0.01,decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

# compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 2)

model.fit(X_train,y_train, batch_size = 48, epochs = 20, callbacks = [es], validation_data = (X_validation, y_validation))

In [ ]:
# checking test accuracy
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
test_acc

In [28]:
# saving files to be loaded in Hyperas functions
np.save("count_vectors.npy", count_vectors.toarray(), allow_pickle=True, fix_imports=True)

table.to_csv("table.csv")

In [90]:
# Tuning hyperparameter with Hyperas
# Code source: https://github.com/maxpumperla/hyperas

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    count_vectors = np.load("count_vectors.npy")
    
    table = pd.read_csv("table.csv")
    table.drop(columns = "Unnamed: 0",inplace=True)
    
    x_train, x_test, y_train, y_test = train_test_split(count_vectors,table.iloc[:,1].values,test_size = 0.2)

    x_train = x_train.reshape(631,755,1)
    y_train = y_train.reshape(631,1)
    x_test = x_test.reshape(158,755,1)
    y_test = y_test.reshape(158,1)
    
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()

    model.add(LSTM({{choice([32,64,96,128])}},activation={{choice(["relu","elu"])}}, 
                   input_shape = (x_train.shape[1:]), return_sequences=True, dropout={{uniform(0,1)}}))
    model.add(BatchNormalization())

    model.add(LSTM({{choice([32,64,96,128])}}, activation={{choice(["relu","elu"])}}, dropout={{uniform(0,1)}}))
    model.add(BatchNormalization())

    model.add(Dense({{choice([32,64,96,128])}}, activation={{choice(["relu","elu"])}}))
    model.add(Dropout({{uniform(0,1)}}))

    model.add(Dense(2, activation="sigmoid"))

    # setting up optimizer hyperparameters
    sgd = SGD(lr={{choice([1e-06,1e-05,1e-04,1e-03,1e-02])}},decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

    # compile model
    model.compile(loss="sparse_categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

    es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 2)

    result = model.fit(x_train,y_train, batch_size = {{choice([16, 32, 64])}}, epochs = {{choice([5, 10, 15])}}, 
                       callbacks = [es], validation_split=0.1)

    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [91]:
if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='Capstone modelling stage v.3-RNN')
    X_train, Y_train, X_test, Y_test = data()
    print("Evaluation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    import warnings
except:
    pass

try:
    from sklearn import pipeline, preprocessing
except:
    pass

try:
    from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
except:
    pass

try:
    from sklearn.metrics import classification_report
except:
    pass

try:
    from skl

448/504 [=========================>....]           
 - ETA: 4s - loss: 0.7094 - acc: 0.4710            
                                                   
480/504 [===========================>..]           
 - ETA: 1s - loss: 0.7143 - acc: 0.4583            
                                                   
504/504 [==============================]           
 - 43s 85ms/step - loss: 0.7138 - acc: 0.4524 - val_loss: 0.6933 - val_acc: 0.4488

Epoch 2/5                                          
 32/504 [>.............................]           
 - ETA: 31s - loss: 0.7061 - acc: 0.6250           
                                                  
 64/504 [==>...........................]           
 - ETA: 29s - loss: 0.7004 - acc: 0.5625           
                                                  
 96/504 [====>.........................]           
 - ETA: 27s - loss: 0.6940 - acc: 0.5729           
                                                  
128/504 [======>...................

176/504 [=========>....................]                                       
 - ETA: 54s - loss: 0.7009 - acc: 0.5511                                       
                                                                              
192/504 [==========>...................]                                       
 - ETA: 51s - loss: 0.7005 - acc: 0.5521                                       
                                                                              
208/504 [===========>..................]                                       
 - ETA: 47s - loss: 0.6994 - acc: 0.5385                                       
                                                                              
224/504 [============>.................]                                       
 - ETA: 44s - loss: 0.6990 - acc: 0.5536                                       
                                                                              
240/504 [=============>................]    

208/504 [===========>..................]                                       
 - ETA: 36s - loss: 0.6868 - acc: 0.5769                                       
                                                                              
224/504 [============>.................]                                       
 - ETA: 34s - loss: 0.6878 - acc: 0.5625                                       
                                                                              
240/504 [=============>................]                                       
 - ETA: 33s - loss: 0.6883 - acc: 0.5583                                       
                                                                              
256/504 [==============>...............]                                       
 - ETA: 30s - loss: 0.6916 - acc: 0.5547                                       
                                                                              
272/504 [===============>..............]    

240/504 [=============>................]                                       
 - ETA: 32s - loss: 0.6976 - acc: 0.5125                                       
                                                                              
256/504 [==============>...............]                                       
 - ETA: 30s - loss: 0.6965 - acc: 0.5117                                       
                                                                              
272/504 [===============>..............]                                       
 - ETA: 28s - loss: 0.6959 - acc: 0.5147                                       
                                                                              
288/504 [================>.............]                                       
 - ETA: 26s - loss: 0.6945 - acc: 0.5208                                       
                                                                              
304/504 [=================>............]    

...

 64/504 [==>...........................]                                      
 - ETA: 31s - loss: 0.6930 - acc: 0.4688                                      
                                                                             
 96/504 [====>.........................]                                      
 - ETA: 28s - loss: 0.6894 - acc: 0.5208                                      
                                                                             
128/504 [======>.......................]                                      
 - ETA: 26s - loss: 0.6946 - acc: 0.4766                                      
                                                                             
160/504 [========>.....................]                                      
 - ETA: 24s - loss: 0.6991 - acc: 0.4875                                      
                                                                             
192/504 [==========>...................]                

 - ETA: 27s - loss: 0.6929 - acc: 0.5428                                      
                                                                             
320/504 [==================>...........]                                      
 - ETA: 25s - loss: 0.6908 - acc: 0.5406                                      
                                                                             
336/504 [===================>..........]                                      
 - ETA: 23s - loss: 0.6907 - acc: 0.5387                                      
                                                                             
352/504 [===================>..........]                                      
 - ETA: 20s - loss: 0.6900 - acc: 0.5455                                      
                                                                             
368/504 [====================>.........]                                      
 - ETA: 18s - loss: 0.6900 - acc: 0.5516                

                                                                             
352/504 [===================>..........]                                      
 - ETA: 18s - loss: 0.6943 - acc: 0.5170                                      
                                                                             
368/504 [====================>.........]                                      
 - ETA: 16s - loss: 0.6959 - acc: 0.5163                                      
                                                                             
384/504 [=====================>........]                                      
 - ETA: 14s - loss: 0.6951 - acc: 0.5182                                      
                                                                             
400/504 [======================>.......]                                      
 - ETA: 12s - loss: 0.6960 - acc: 0.5175                                      
                                                        

384/504 [=====================>........]                                      
 - ETA: 14s - loss: 0.6962 - acc: 0.5208                                      
                                                                             
400/504 [======================>.......]                                      
 - ETA: 12s - loss: 0.6961 - acc: 0.5225                                      
                                                                             
416/504 [=======================>......]                                      
 - ETA: 10s - loss: 0.6957 - acc: 0.5192                                      
                                                                             
432/504 [========================>.....]                                      
 - ETA: 8s - loss: 0.6948 - acc: 0.5278                                       
                                                                              
448/504 [=========================>....]               

 - ETA: 16s - loss: 0.6857 - acc: 0.5938                                      
                                                                             
128/504 [======>.......................]                                      
 - ETA: 13s - loss: 0.6929 - acc: 0.5625                                      
                                                                             
192/504 [==========>...................]                                      
 - ETA: 11s - loss: 0.6973 - acc: 0.5469                                      
                                                                             
256/504 [==============>...............]                                      
 - ETA: 9s - loss: 0.6991 - acc: 0.5273                                       
                                                                              
320/504 [==================>...........]                                      
 - ETA: 6s - loss: 0.6955 - acc: 0.5281                